<a href="https://colab.research.google.com/github/Juaano28/Parcial02_TAM/blob/main/PARCIAL_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parcial 2 TAM


*   Nicolás Castaño Pérez
*   Juan Esteban López Marin



In [ ]:
!pip install scikit-optimize
!pip install streamlit -q #instalación de librerías
!pip install pyngrok
!pip install optuna
!pip install streamlit pandas matplotlib seaborn scikit-learn pyngrok kagglehub
!pip install pyngrok streamlit --quiet

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import h5py
with h5py.File('/content/drive/Shareddrives/UNAL_Colab/Teoría de Aprendizaje de Máquina/Parcial_2/usps.h5', 'r') as hf:
        train = hf.get('train')
        X_tr = train.get('data')[:]
        y_tr = train.get('target')[:]
        test = hf.get('test')
        X_te = test.get('data')[:]
        y_te = test.get('target')[:]


##  Punto (b) – Proyección USPS con PCA y UMAP

###  Objetivo
Este análisis busca proyectar el conjunto de datos USPS a un espacio bidimensional usando dos técnicas de reducción de dimensionalidad: **PCA** (Análisis de Componentes Principales) y **UMAP** (Uniform Manifold Approximation and Projection). Además, se explora cómo varía la representación obtenida al modificar el parámetro `n_neighbors` en UMAP.

---

###  Comparación entre PCA y UMAP

- **PCA** es un método lineal que proyecta los datos en las direcciones que maximizan la varianza global. En la proyección obtenida:
  - Se observa cierta agrupación por dígito.
  - Sin embargo, hay bastante **solapamiento entre clases**, especialmente entre dígitos similares como el 3 y el 8.
  - La estructura de los datos no se representa bien cuando hay no linealidades o múltiples variedades locales.

- **UMAP**, en cambio, es un método no lineal que busca preservar tanto la **estructura local** como la **estructura global** del conjunto de datos:
  - Produce **clústeres más compactos y claramente separados**.
  - El agrupamiento se alinea mejor con las etiquetas reales de los dígitos.
  - Se logra una representación más rica de la estructura latente.

---

###  Efecto del parámetro `n_neighbors` en UMAP

Se exploraron cuatro valores: **5, 15, 50, 100**.

- **n_neighbors = 5**:
  - Alta preservación local, los clústeres son muy compactos.
  - Riesgo de fragmentación (más ruido visual).

- **n_neighbors = 15**:
  - Equilibrio entre estructura local y global.
  - Clústeres bien definidos con suficiente separación.

- **n_neighbors = 50**:
  - Los clústeres tienden a ser más grandes y más difusos.
  - Más coherencia global, pero menos detalle local.

- **n_neighbors = 100**:
  - Predomina la estructura global, se pierde separación entre clases similares.
  - Posible fusión de clases cercanas.

---

###  Conclusión

UMAP ofrece una proyección **más adecuada para visualización y análisis exploratorio** en comparación con PCA, especialmente cuando se requiere preservar relaciones no lineales entre muestras. Además, el parámetro `n_neighbors` es crucial: valores bajos priorizan detalles locales, mientras que valores altos promueven una estructura más global pero menos precisa por clase.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
import umap

# Verifica que X_tr y y_tr estén definidos previamente
# Ejemplo (si tienes los datos guardados):
# X_tr = np.load("usps_X.npy")
# y_tr = np.load("usps_y.npy")

# Mostrar dimensiones del conjunto de datos
print(f"USPS (HDF5): {X_tr.shape[0]} muestras, {X_tr.shape[1]} dimensiones")

# Asegurar que las etiquetas sean enteros
y_tr = y_tr.astype(int)

# --------------------------
# Proyección PCA a 2D
# --------------------------
pca = PCA(n_components=2, random_state=0)
X_pca = pca.fit_transform(X_tr)

# --------------------------
# Proyección UMAP a 2D con configuración base
# --------------------------
umap_model = umap.UMAP(
    n_components=2,
    n_neighbors=15,
    min_dist=0.1,
    metric="euclidean",
    random_state=0
)
X_umap = umap_model.fit_transform(X_tr)

# --------------------------
# Función de visualización con imágenes superpuestas
# --------------------------
def plot_embedding(X_emb, title, show_images=True, sample_images=100, ax=None):
    if ax is None:
        plt.figure(figsize=(8, 8))
        ax = plt.gca()

    # Crear un DataFrame para visualización
    df = pd.DataFrame({
        "x": X_emb[:, 0],
        "y": X_emb[:, 1],
        "label": y_tr
    })

    # Graficar los puntos coloreados por etiqueta
    sns.scatterplot(
        data=df,
        x="x", y="y",
        hue="label",
        palette="tab10",
        s=5, alpha=0.6,
        legend="full",
        ax=ax
    )
    ax.set_title(title)
    ax.legend(title="Dígito", loc='best')
    ax.grid(True)

    # Superponer imágenes representativas (opcional)
    if show_images:
        indices = np.random.choice(len(X_tr), size=sample_images, replace=False)
        for idx in indices:
            xi, yi = X_emb[idx]
            img = X_tr[idx].reshape(16, 16)
            ax.imshow(
                img, cmap="gray",
                extent=(xi-0.5, xi+0.5, yi-0.5, yi+0.5),
                alpha=0.7
            )

# --------------------------
# Visualización PCA vs UMAP base
# --------------------------
plt.figure(figsize=(16, 8))

# PCA
ax1 = plt.subplot(1, 2, 1)
plot_embedding(X_pca, "Proyección PCA 2D – USPS", ax=ax1)

# UMAP (n_neighbors=15)
ax2 = plt.subplot(1, 2, 2)
plot_embedding(X_umap, "Proyección UMAP 2D (n_neighbors=15) – USPS", ax=ax2)

plt.tight_layout()
plt.show()

# --------------------------
# Comparación UMAP con diferentes n_neighbors
# --------------------------
plt.figure(figsize=(12, 6))
neighbors = [5, 15, 50, 100]

for i, nn in enumerate(neighbors):
    # Proyectar con UMAP para cada valor de n_neighbors
    emb = umap.UMAP(
        n_components=2,
        n_neighbors=nn,
        min_dist=0.1,
        metric="euclidean",
        random_state=0
    ).fit_transform(X_tr)

    # Visualizar sin superposición de imágenes
    ax = plt.subplot(2, 2, i + 1)
    plot_embedding(emb, f"UMAP n_neighbors={nn}", show_images=False, ax=ax)

plt.tight_layout()
plt.show()


# C)

##  Punto (c) – Clasificación con 3 modelos

Se evaluaron tres modelos de clasificación sobre tres espacios distintos de representación: el conjunto **original** de datos USPS, y sus proyecciones usando **PCA (20 componentes)** y **UMAP (20 dimensiones)**.

Los modelos elegidos son:

---

###  1. Regresión Logística (Logistic Regression)

- **Justificación**:
  - Es un modelo lineal probabilístico muy eficiente para clasificación multiclase.
  - Sirve como línea base robusta para comparar otros modelos.
  - Su interpretación es clara y no requiere mucha afinación.

- **Hiperparámetros usados**:
  - `solver='lbfgs'`: optimizador eficiente para clasificación multiclase.
  - `multi_class='multinomial'`: modela directamente la probabilidad multiclase (mejor que one-vs-rest).
  - `max_iter=1000`: asegura convergencia con datos proyectados de alta dimensionalidad.

- **Observaciones**:
  - Se desempeñó de forma razonable, especialmente en datos proyectados por PCA.
  - Sensible a relaciones no lineales, lo que limita su rendimiento en UMAP o datos originales.

---

###  2. Random Forest Classifier

- **Justificación**:
  - Modelo robusto y no paramétrico basado en árboles.
  - Captura relaciones no lineales y es poco sensible a outliers.
  - Ofrece alta precisión sin requerir normalización de datos.

- **Hiperparámetros usados**:
  - `n_estimators=100`: 100 árboles para garantizar estabilidad.
  - `max_depth=20`: limita la profundidad para evitar sobreajuste.

- **Observaciones**:
  - Buen desempeño tanto en el espacio original como en UMAP.
  - Aprovecha la variabilidad de la representación para mejorar la clasificación.
  - Más lento que la regresión logística, pero más preciso en estructuras complejas.

---

###  3. Red Neuronal Profunda (MLP)

- **Justificación**:
  - Permite modelar relaciones complejas y no lineales entre las variables.
  - Capaz de adaptarse a representaciones densas como UMAP o PCA.
  - Permite obtener probabilidades suaves para cada clase → útil para ROC.

- **Arquitectura**:
  - `Dense(128, relu) → Dropout(0.3) → Dense(64, relu) → Dense(10, softmax)`
  - Dos capas ocultas con activación ReLU y una capa de salida softmax para clasificación multiclase.

- **Hiperparámetros usados**:
  - `epochs=20`: entrenamiento corto para evitar sobreajuste.
  - `batch_size=64`: tamaño de lote equilibrado.
  - `Dropout(0.3)`: regularización que previene overfitting.
  - `optimizer=Adam(learning_rate=0.001)`: adaptativo y eficiente para tareas multicategoría.

- **Observaciones**:
  - Modelo más costoso en tiempo de entrenamiento.
  - Excelente rendimiento sobre datos proyectados con UMAP, al capturar mejor las relaciones complejas.

---

###  Evaluación

- Se utilizó `train_test_split` con 20% de test para todos los modelos.
- Se midió `accuracy`, `classification_report` y se graficó la **curva ROC multiclase** para cada clasificador.
- Las curvas ROC permiten comparar la capacidad discriminativa del modelo en cada clase.

---

###  Conclusión

Los tres modelos permiten evaluar desde soluciones lineales simples hasta clasificadores no lineales y complejos. Los resultados muestran que:
- **PCA** mejora ligeramente el desempeño de modelos lineales.
- **UMAP** ofrece mejores resultados para modelos como Random Forest y redes neuronales.
- La **red neuronal** sobresale en espacios no lineales y ofrece predicciones más suaves y calibradas.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
import umap
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import label_binarize, StandardScaler
from sklearn.metrics import classification_report, accuracy_score, roc_curve, auc
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import h5py

# Cargar datos USPS
with h5py.File('/content/drive/Shareddrives/UNAL_Colab/Teoría de Aprendizaje de Máquina/Parcial_2/usps.h5', 'r') as hf:
    X = hf['train']['data'][:]
    y = hf['train']['target'][:].astype(int)

# Proyecciones
X_pca = PCA(n_components=20).fit_transform(X)
X_umap = umap.UMAP(n_components=20, n_neighbors=15, random_state=0).fit_transform(X)

# Datos para clasificación
datasets = {
    "Original": X,
    "PCA (20)": X_pca,
    "UMAP (20)": X_umap
}

# Clasificadores clásicos
def evaluate_classifier(clf, X_train, X_test, y_train, y_test, title):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f"=== {title} ===")
    print(f"Accuracy: {acc:.4f}")
    print(classification_report(y_test, y_pred))

    # ROC multiclase
    y_test_bin = label_binarize(y_test, classes=np.unique(y))
    y_score = clf.predict_proba(X_test)
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(10):
        fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    plt.figure(figsize=(8, 6))
    for i in range(10):
        plt.plot(fpr[i], tpr[i], label=f"Clase {i} (AUC = {roc_auc[i]:.2f})")
    plt.plot([0, 1], [0, 1], 'k--')
    plt.title(f"Curva ROC - {title}")
    plt.xlabel("FPR")
    plt.ylabel("TPR")
    plt.legend()
    plt.grid(True)
    plt.show()

# Clasificador simple basado en Deep Learning
def evaluate_nn(X, y, title):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    y_train_cat = to_categorical(y_train, 10)
    y_test_cat = to_categorical(y_test, 10)

    model = Sequential([
        Dense(128, activation='relu', input_shape=(X.shape[1],)),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dense(10, activation='softmax')
    ])

    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(X_train, y_train_cat, validation_split=0.1, epochs=20, batch_size=64, verbose=0)

    loss, acc = model.evaluate(X_test, y_test_cat, verbose=0)
    print(f"=== NN - {title} ===")
    print(f"Accuracy: {acc:.4f}")

    # ROC multiclase
    y_pred_prob = model.predict(X_test)
    fpr, tpr, roc_auc = {}, {}, {}
    for i in range(10):
        fpr[i], tpr[i], _ = roc_curve(y_test_cat[:, i], y_pred_prob[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    plt.figure(figsize=(8, 6))
    for i in range(10):
        plt.plot(fpr[i], tpr[i], label=f"Clase {i} (AUC = {roc_auc[i]:.2f})")
    plt.plot([0, 1], [0, 1], 'k--')
    plt.title(f"Curva ROC - NN {title}")
    plt.xlabel("FPR")
    plt.ylabel("TPR")
    plt.legend()
    plt.grid(True)
    plt.show()

# Evaluar cada conjunto proyectado
for name, X_proj in datasets.items():
    print(f"\n\n--- Evaluación: {name} ---")
    X_train, X_test, y_train, y_test = train_test_split(X_proj, y, test_size=0.2, random_state=42)

    # Logistic Regression
    lr = LogisticRegression(max_iter=1000, solver='lbfgs', multi_class='multinomial')
    evaluate_classifier(lr, X_train, X_test, y_train, y_test, f"LogReg - {name}")

    # Random Forest
    rf = RandomForestClassifier(n_estimators=100, max_depth=20, random_state=0)
    evaluate_classifier(rf, X_train, X_test, y_train, y_test, f"RF - {name}")

    # Red neuronal simple
    evaluate_nn(X_proj, y, f"{name}")


# d)

In [ ]:
import pickle
import os
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import numpy as np
import h5py
from sklearn.decomposition import PCA
import umap

# Define the directory to save the models
model_dir = '/content/drive/Shareddrives/UNAL_Colab/Teoría de Aprendizaje de Máquina/Parcial_2/trained_models'
os.makedirs(model_dir, exist_ok=True)

# Load data explicitly
data_path = '/content/drive/Shareddrives/UNAL_Colab/Teoría de Aprendizaje de Máquina/Parcial_2/usps.h5'
with h5py.File(data_path, 'r') as hf:
    train = hf.get('train')
    X_tr = train.get('data')[:]
    y_tr = train.get('target')[:]
    test = hf.get('test')
    X_te = test.get('data')[:]
    y_te = test.get('target')[:]
X = np.vstack((X_tr, X_te))
y = np.hstack((y_tr, y_te)).astype(int)


# Perform Dimensionality Reduction (using 20 components for classification as in point c)
pca_20d = PCA(n_components=20, random_state=0)
X_pca_20d = pca_20d.fit_transform(X)

umap_20d = umap.UMAP(n_components=20, n_neighbors=15, min_dist=0.1, metric="euclidean", random_state=0)
X_umap_20d = umap_20d.fit_transform(X)

# Create datasets dictionary
datasets_to_save = {
    "Original": X,
    "PCA_20": X_pca_20d,
    "UMAP_20": X_umap_20d # Corrected variable name here
}

# Train and save models
for name, X_data in datasets_to_save.items():
    print(f"Training models on {name} dataset...")
    # Split data for training (use the same split as in the dashboard for consistency)
    X_train, X_test, y_train, y_test = train_test_split(X_data, y, test_size=0.2, random_state=42)

    # Train Logistic Regression
    lr_model = LogisticRegression(max_iter=1000, solver='lbfgs', multi_class='multinomial')
    lr_model.fit(X_train, y_train)
    lr_filename = os.path.join(model_dir, f'lr_model_{name}.pickle')
    with open(lr_filename, 'wb') as f:
        pickle.dump(lr_model, f)
    print(f"Saved Logistic Regression model for {name} to {lr_filename}")

    # Train Random Forest
    rf_model = RandomForestClassifier(n_estimators=100, max_depth=20, random_state=0)
    rf_model.fit(X_train, y_train)
    rf_filename = os.path.join(model_dir, f'rf_model_{name}.pickle')
    with open(rf_filename, 'wb') as f:
        pickle.dump(rf_model, f)
    print(f"Saved Random Forest model for {name} to {rf_filename}")

    # Train Neural Network
    # Need to adjust input shape for the NN
    input_dim = X_train.shape[1]
    nn_model = Sequential([
        Dense(128, activation='relu', input_shape=(input_dim,)),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dense(10, activation='softmax')
    ])
    nn_model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    nn_model.fit(X_train, to_categorical(y_train, 10), validation_split=0.1, epochs=20, batch_size=64, verbose=0) # Train NN

    nn_filename = os.path.join(model_dir, f'nn_model_{name}.h5') # Save NN model in HDF5 format
    nn_model.save(nn_filename)
    print(f"Saved Neural Network model for {name} to {nn_filename}")

print("\nAll models trained and saved successfully.")

In [ ]:
%%writefile usps_dashboard.py

import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
import umap
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import label_binarize, StandardScaler
from sklearn.metrics import classification_report, accuracy_score, roc_curve, auc
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, load_model # Import load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import h5py
import io
import pickle # Import pickle
import os.path # Explicitly import os.path

# Define the directory where models are saved
model_dir = '/content/drive/Shareddrives/UNAL_Colab/Teoría de Aprendizaje de Máquina/Parcial_2/trained_models'


# Título del Dashboard
st.title("Análisis de Datos USPS con Reducción de Dimensionalidad y Clasificación")

st.markdown("""
Este dashboard interactivo permite explorar la proyección del conjunto de datos de dígitos escritos a mano USPS utilizando
técnicas de reducción de dimensionalidad como PCA y UMAP. Además, evalúa el rendimiento de diferentes clasificadores
(Regresión Logística, Random Forest y Red Neuronal) en los datos originales y en los espacios de menor dimensión.

Utilice la barra lateral para configurar los parámetros de visualización y clasificación.
""")


# Cargar datos
@st.cache_data
def load_usps_data(filepath):
    """Loads the USPS dataset from an HDF5 file."""
    with h5py.File(filepath, 'r') as hf:
        train = hf.get('train')
        X_tr = train.get('data')[:]
        y_tr = train.get('target')[:]
        test = hf.get('test')
        X_te = test.get('data')[:]
        y_te = test.get('target')[:]
    return X_tr, y_tr, X_te, y_te

data_path = '/content/drive/Shareddrives/UNAL_Colab/Teoría de Aprendizaje de Máquina/Parcial_2/usps.h5'
X_tr, y_tr, X_te, y_te = load_usps_data(data_path)
X = np.vstack((X_tr, X_te))
y = np.hstack((y_tr, y_te))

y = y.astype(int) # Ensure labels are integers

# --- Dimensionality Reduction Functions ---
@st.cache_data
def perform_pca(X, n_components):
    """Performs Principal Component Analysis (PCA) on the data."""
    st.info(f"Aplicando PCA con {n_components} componentes.")
    pca = PCA(n_components=n_components, random_state=0)
    X_pca = pca.fit_transform(X)
    return X_pca

@st.cache_data
def perform_umap(X, n_components, n_neighbors, min_dist, metric):
    """Performs Uniform Manifold Approximation and Projection (UMAP) on the data."""
    st.info(f"Aplicando UMAP con {n_components} componentes, n_neighbors={n_neighbors}, min_dist={min_dist}, metric='{metric}'.")
    umap_model = umap.UMAP(
        n_components=n_components,
        n_neighbors=n_neighbors, # Controls local vs global structure
        min_dist=min_dist,       # Controls how tightly points are clustered together
        metric=metric,           # Distance metric
        random_state=0
    )
    X_umap = umap_model.fit_transform(X)
    return X_umap

# --- Visualization Function ---
def plot_embedding(X_emb, y, title, show_images=False, sample_images=100, X_original=None):
    """Plots the 2D data embedding, optionally with superimposed images."""
    fig, ax = plt.subplots(figsize=(10, 8))

    df = pd.DataFrame({
        "x": X_emb[:, 0],
        "y": X_emb[:, 1],
        "label": y
    })

    sns.scatterplot(
        data=df,
        x="x", y="y",
        hue="label",
        palette="tab10", # A good default categorical palette
        s=10, alpha=0.7, # Point size and transparency
        legend="full",
        ax=ax
    )
    ax.set_title(title)
    ax.legend(title="Dígito", loc='best')
    ax.grid(True)

    # Superponer imágenes representativas (opcional) solo si es 2D
    if show_images and X_original is not None and X_emb.shape[1] == 2:
        # Use a fixed random state for reproducibility in image sampling
        np.random.seed(0)
        indices = np.random.choice(len(X_original), size=sample_images, replace=False)
        for idx in indices:
            xi, yi = X_emb[idx]
            img = X_original[idx].reshape(16, 16) # Reshape the 256 features back to 16x16 image

            # Adjust extent based on your data scale if needed for better visual
            # The extent should match the approximate scale of your projected points.
            # This is a heuristic, might need adjustment based on actual projection ranges.
            # Calculate rough scale of the embedding for image sizing
            x_range = np.max(X_emb[:, 0]) - np.min(X_emb[:, 0])
            y_range = np.max(X_emb[:, 1]) - np.min(X_emb[:, 1])
            img_size_factor = 0.015 # Adjust this factor to change image size relative to plot size

            ax.imshow(
                img, cmap="gray", # Grayscale colormap for digits
                extent=(xi - x_range * img_size_factor, xi + x_range * img_size_factor,
                        yi - y_range * img_size_factor, yi + y_range * img_size_factor),
                alpha=0.8, # Slightly more opaque images
                aspect='auto', # Allow aspect ratio to be determined by extent
                interpolation='bilinear' # Smooth out pixelated images
            )


    st.pyplot(fig)
    plt.close(fig) # Close the figure to free memory

def plot_roc_curve(fpr_dict, tpr_dict, roc_auc_dict, title):
    """Plots the multi-class ROC curve."""
    fig, ax = plt.subplots(figsize=(8, 6))
    for i in range(10):
        if i in fpr_dict and fpr_dict[i] is not None: # Check if class exists and data is not None
            ax.plot(fpr_dict[i], tpr_dict[i], label=f"Clase {i} (AUC = {roc_auc_dict[i]:.2f})")
        else:
             # Plot a minimal line or skip if no data for this class
             ax.plot([], [], label=f"Clase {i} (No data)") # Add empty plot for missing classes in legend

    ax.plot([0, 1], [0, 1], 'k--', label='Aleatorio (AUC = 0.50)') # Add diagonal line for random chance
    ax.set_title(f"Curva ROC - {title}")
    ax.set_xlabel("Tasa de Falsos Positivos (FPR)")
    ax.set_ylabel("Tasa de Verdaderos Positivos (TPR)")
    ax.legend(loc='lower right') # Position legend
    ax.grid(True)
    st.pyplot(fig)
    plt.close(fig) # Close the figure to free memory


# --- Classification Functions ---
# Load models instead of evaluating classifiers
@st.cache_resource # Use cache_resource for models as they are large objects
def load_classifier_model(model_name, dataset_name):
    """Loads a pre-trained scikit-learn classifier model."""
    model_path = os.path.join(model_dir, f'{model_name}_model_{dataset_name}.pickle')
    if os.path.exists(model_path):
        st.info(f"Cargando modelo: {model_name} en dataset: {dataset_name}")
        with open(model_path, 'rb') as f:
            model = pickle.load(f)
        return model
    st.warning(f"Modelo {model_name} para {dataset_name} no encontrado en {model_path}")
    return None

@st.cache_resource # Use cache_resource for models
def load_nn_model(dataset_name):
    """Loads a pre-trained Keras Neural Network model."""
    model_path = os.path.join(model_dir, f'nn_model_{dataset_name}.h5')
    if os.path.exists(model_path):
        st.info(f"Cargando modelo: Neural Network en dataset: {dataset_name}")
        model = load_model(model_path)
        return model
    st.warning(f"Modelo Neural Network para {dataset_name} no encontrado en {model_path}")
    return None

# Function to get classification report and ROC data from loaded models
def get_classification_metrics(model, X_test, y_test, y_all_classes):
    """Calculates classification metrics and ROC data for a given model."""
    if model is None:
        return None, None, {}, {}, {} # Return empty data if model is None

    # Get predicted class labels
    if hasattr(model, 'predict_proba'):
        y_pred_prob = model.predict_proba(X_test)
        y_pred = model.predict(X_test) # Use predict for scikit-learn models
    elif hasattr(model, 'predict'): # For Neural Networks
        y_pred_prob = model.predict(X_test)
        y_pred = np.argmax(y_pred_prob, axis=1) # Get class with highest probability for Keras models
    else:
        st.error("Model does not have 'predict_proba' or 'predict' method.")
        return None, None, {}, {}, {}


    acc = accuracy_score(y_test, y_pred)

    # Convert report to DataFrame for better display in Streamlit
    report_dict = classification_report(y_test, y_pred, output_dict=True)
    report_df = pd.DataFrame(report_dict).transpose()


    # ROC multiclase
    y_test_bin = label_binarize(y_test, classes=np.unique(y_all_classes))

    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    if y_pred_prob is not None:
        n_classes = len(np.unique(y_all_classes))
        # Ensure y_score has the correct shape (n_samples, n_classes)
        if y_pred_prob.shape[1] == n_classes:
            for i in np.unique(y_all_classes):
                if i in np.unique(y_test):
                    try:
                         # Ensure y_test_bin[:, i] is for the current class and y_score[:, i] are probabilities
                         fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_pred_prob[:, i])
                         roc_auc[i] = auc(fpr[i], tpr[i])

                    except Exception as e:
                         #st.warning(f"Could not calculate ROC for class {i}: {e}") # Avoid excessive warnings in UI
                         fpr[i], tpr[i], roc_auc[i] = None, None, None # Assign None if calculation fails
                else:
                    # Class not present in the test set, cannot calculate ROC
                    fpr[i], tpr[i], roc_auc[i] = None, None, None # Assign None if class not in test set
        else:
            st.warning(f"Mismatch in y_score shape ({y_pred_prob.shape}) and number of classes ({n_classes}). Cannot calculate ROC.")
            # Ensure empty ROC data is returned
            for i in np.unique(y_all_classes):
                 fpr[i], tpr[i], roc_auc[i] = None, None, None


    return acc, report_df, fpr, tpr, roc_auc


# --- Streamlit App Layout ---
st.sidebar.header("Configuración")
analysis_mode = st.sidebar.radio("Seleccione el Modo de Análisis", ["Proyección 2D", "Clasificación"])

if analysis_mode == "Proyección 2D":
    st.header("Proyección a 2D (Visualización)")

    st.markdown("""
    Seleccione una técnica de reducción de dimensionalidad para visualizar el conjunto de datos USPS en 2 dimensiones.
    Esto ayuda a entender la estructura y separabilidad de las clases.
    """)

    projection_method = st.sidebar.selectbox("Método de Proyección 2D", ["PCA", "UMAP"])

    if projection_method == "PCA":
        st.subheader("Proyección PCA a 2D")
        st.markdown("""
        **PCA (Análisis de Componentes Principales)** es un método lineal que proyecta los datos
        en las direcciones de máxima varianza. Es útil para visualizar las diferencias globales
        entre clases.
        """)
        X_pca_2d = perform_pca(X, n_components=2)
        plot_embedding(X_pca_2d, y, "Proyección PCA 2D – USPS", show_images=True, X_original=X)

    elif projection_method == "UMAP":
        st.subheader("Proyección UMAP a 2D")
        st.markdown("""
        **UMAP (Uniform Manifold Approximation and Projection)** es un método no lineal que
        busca preservar tanto la estructura local como la global. A menudo produce clústeres
        más compactos y separados que PCA, siendo ideal para visualizaciones donde las clases
        tienen fronteras no lineales.
        """)

        st.sidebar.subheader("Parámetros de UMAP 2D")
        n_neighbors_umap = st.sidebar.slider(
            "n_neighbors",
            5, 100, 15,
            help="Número de vecinos a considerar para construir el gráfico del co-vecindario. Valores bajos (<10) enfatizan la estructura local; valores altos (>50) enfatizan la estructura global."
        )
        min_dist_umap = st.sidebar.slider(
            "min_dist",
            0.0, 1.0, 0.1, 0.05,
            help="Controla la distancia mínima entre puntos en el espacio de baja dimensión. Valores bajos resultan en clústeres más compactos; valores altos resultan en una dispersión más laxa."
        )
        metric_umap = st.sidebar.selectbox(
            "Metric",
            ["euclidean", "manhattan", "cosine"],
            help="Métrica de distancia para construir el gráfico del co-vecindario en el espacio original."
        )


        X_umap_2d = perform_umap(X, n_components=2, n_neighbors=n_neighbors_umap, min_dist=min_dist_umap, metric=metric_umap)
        plot_embedding(X_umap_2d, y, f"Proyección UMAP 2D (n_neighbors={n_neighbors_umap}, min_dist={min_dist_umap}) – USPS", show_images=True, X_original=X)

        st.subheader("Comparación UMAP con diferentes n_neighbors")
        st.markdown("""
        Esta sección muestra cómo la proyección UMAP varía al cambiar el parámetro `n_neighbors`.
        Observe cómo valores bajos tienden a "fragmentar" la visualización (priorizando la estructura local),
        mientras que valores altos pueden fusionar clústeres (priorizando la estructura global).
        """)
        neighbors_compare = [5, 15, 50, 100]
        cols = st.columns(2) # Use columns for better layout

        for i, nn in enumerate(neighbors_compare):
            col = cols[i % 2]
            with col:
                st.write(f"**n_neighbors = {nn}**")
                # Add description based on n_neighbors value
                if nn == 5:
                    st.write("_Enfatiza la estructura local, clústeres muy compactos, posible fragmentación._")
                elif nn == 15:
                     st.write("_Equilibrio entre estructura local y global, clústeres bien definidos._")
                elif nn == 50:
                     st.write("_Más coherencia global, clústeres más difusos, menos detalle local._")
                elif nn == 100:
                     st.write("_Predomina la estructura global, posible fusión de clases similares._")

                X_umap_nn = perform_umap(X, n_components=2, n_neighbors=nn, min_dist=0.1, metric="euclidean")
                plot_embedding(X_umap_nn, y, f"UMAP n_neighbors={nn}", show_images=False) # No images in comparison plots


elif analysis_mode == "Clasificación":
    st.header("Evaluación de Clasificadores")

    st.markdown("""
    Esta sección evalúa el rendimiento de diferentes modelos de clasificación
    en el conjunto de datos original y en las proyecciones obtenidas por PCA y UMAP.
    Los modelos fueron pre-entrenados y cargados para su evaluación.
    """)

    st.sidebar.subheader("Configuración de Clasificación")
    n_components_dr = st.sidebar.slider(
        "Componentes para DR (Clasificación)",
        2, 256, 20,
        help="Número de componentes a mantener después de aplicar PCA o UMAP para la clasificación."
    )
    classifier_method = st.sidebar.selectbox(
        "Clasificador",
        ["Logistic Regression", "Random Forest", "Neural Network"],
        help="Seleccione el modelo de clasificación a evaluar."
    )

    st.subheader(f"Resultados de Clasificación con {classifier_method} (n_components={n_components_dr})")

    # Perform DR for classification (need to ensure consistency with how models were trained)
    # We need to apply the same DR to the full dataset and then split for test
    # Add information about the DR being performed for classification
    st.info(f"Aplicando Reducción de Dimensionalidad ({n_components_dr} componentes) a los datos completos antes de la división test/train para la evaluación.")

    X_pca_dr_full = perform_pca(X, n_components=n_components_dr)
    # Using default UMAP params for DR for classification, consistent with training cell
    X_umap_dr_full = perform_umap(X, n_components=n_components_dr, n_neighbors=15, min_dist=0.1, metric="euclidean")

    datasets_for_classification = {
        "Original": X,
        f"PCA ({n_components_dr})": X_pca_dr_full,
        f"UMAP ({n_components_dr})": X_umap_dr_full
    }

    classification_results_dict = {} # Store results for display
    roc_data = {} # Store ROC data for plotting

    # Map classifier method string to the prefix used in roc_data keys (consistent with evaluation cell)
    classifier_prefix_map = {
        "Logistic Regression": "LogReg",
        "Random Forest": "RF",
        "Neural Network": "NN"
    }
    # Get the actual name used for saving models based on the selected classifier
    model_file_prefix_map = {
        "Logistic Regression": "lr",
        "Random Forest": "rf",
        "Neural Network": "nn"
    }
    selected_model_prefix = model_file_prefix_map.get(classifier_method, "")


    for name, X_data in datasets_for_classification.items():
        st.write(f"### Dataset: {name}")
        # Split data for evaluation (using the same random state as training)
        # This ensures we evaluate on the exact same test set used during training
        X_train, X_test, y_train, y_test = train_test_split(X_data, y, test_size=0.2, random_state=42)

        # Load the appropriate model based on the selected classifier and dataset name
        if classifier_method == "Neural Network":
             # For NN, the dataset name used in the filename is slightly different (e.g., "UMAP_20")
             # We need to map the display name to the filename name
             filename_dataset_name = name.replace(" ", "_").replace("(", "").replace(")", "") # Convert "PCA (20)" to "PCA_20" etc.
             model = load_nn_model(filename_dataset_name)
        else:
             # For LR and RF, the dataset name in the filename is also adjusted (e.g., "PCA_20")
             filename_dataset_name = name.replace(" ", "_").replace("(", "").replace(")", "")
             model = load_classifier_model(selected_model_prefix, filename_dataset_name)


        if model:
            st.subheader(f"Métricas de {classifier_method} en {name}")
            acc, report_df, fpr, tpr, roc_auc = get_classification_metrics(model, X_test, y_test, y)

            if acc is not None:
                st.write(f"**Accuracy:** {acc:.4f}")

            if report_df is not None:
                st.write("**Reporte de Clasificación:**")
                st.dataframe(report_df) # Display report as a table

            # Store ROC data with a consistent key structure for plotting
            # Use the display name for clarity in the ROC plot title
            roc_data[f"{classifier_method} - {name}"] = {"fpr": fpr, "tpr": tpr, "roc_auc": roc_auc}
        else:
            st.warning(f"No se pudo cargar el modelo {classifier_method} para el dataset {name}.")
            # Ensure empty ROC data is stored if model not found to avoid errors later
            roc_data[f"{classifier_method} - {name}"] = {"fpr": {}, "tpr": {}, "roc_auc": {}}


    # Plot ROC curves for the selected classifier across different datasets
    st.subheader("Curvas ROC Comparativas")
    st.markdown(f"""
    Estas curvas ROC muestran el rendimiento de la **{classifier_method}**
    para cada clase en los diferentes conjuntos de datos (Original, PCA, UMAP).
    Una curva más cercana a la esquina superior izquierda indica un mejor rendimiento.
    El área bajo la curva (AUC) resume la capacidad discriminativa del clasificador
    para esa clase (1.0 es perfecto, 0.5 es aleatorio).
    """)

    # Filter roc_data based on the selected classifier name (using the display name for filtering)
    filtered_roc_data = {key: value for key, value in roc_data.items() if key.startswith(classifier_method)}


    if filtered_roc_data:
        # Check if there is any valid ROC data to plot after filtering
        # A more robust check: check if any of the filtered entries have non-empty fpr dictionaries
        has_valid_roc_data_to_plot = any(data.get("fpr") and any(v is not None for v in data["fpr"].values()) for key, data in filtered_roc_data.items())

        if has_valid_roc_data_to_plot:
             # Sort keys for consistent plotting order (e.g., Original, PCA, UMAP)
             # This custom sort places 'Original' first, then 'PCA', then 'UMAP'
             def custom_sort_key(item):
                 key = item[0]
                 if "Original" in key:
                     return 0
                 elif "PCA" in key:
                     return 1
                 elif "UMAP" in key:
                     return 2
                 else:
                     return 3 # Other keys come last

             sorted_filtered_items = sorted(filtered_roc_data.items(), key=custom_sort_key)

             for key, data in sorted_filtered_items:
                 if data.get("fpr") and any(v is not None for v in data["fpr"].values()):
                     plot_roc_curve(data["fpr"], data["tpr"], data["roc_auc"], key) # Use the key as the title
                 else:
                     st.info(f"No hay datos completos de Curva ROC disponibles para {key}.")

        else:
             st.info(f"No hay datos de Curva ROC disponibles para el clasificador {classifier_method} en los conjuntos de datos seleccionados.")

    else:
        st.info("No hay datos de Curva ROC disponibles para el clasificador y conjuntos de datos seleccionados.")

    st.subheader("Hiperparámetros Utilizados y Justificación (Clasificadores)")
    st.markdown("""
    Los modelos de clasificación utilizados en este dashboard fueron entrenados con los siguientes hiperparámetros,
    seleccionados para ofrecer un buen balance entre rendimiento y eficiencia:

    *   **Regresión Logística (`LogisticRegression`)**:
        *   `max_iter=1000`: Aumentado a 1000 para asegurar que el modelo converja, especialmente en datos con mayor dimensionalidad o complejidad como las proyecciones.
        *   `solver='lbfgs'`: Optimizador predeterminado y eficiente para conjuntos de datos pequeños y medianos. Funciona bien para clasificación multiclase.
        *   `multi_class='multinomial'`: Configurado para manejar directamente la clasificación multiclase (más de 2 clases) en lugar de usar una estrategia "one-vs-rest".

    *   **Random Forest (`RandomForestClassifier`)**:
        *   `n_estimators=100`: Número de árboles en el bosque. Un valor de 100 es un buen punto de partida que generalmente proporciona estabilidad sin un costo computacional excesivo. Más árboles tienden a mejorar el rendimiento hasta cierto punto.
        *   `max_depth=20`: Profundidad máxima de cada árbol. Limitar la profundidad ayuda a prevenir el sobreajuste a los datos de entrenamiento, manteniendo la capacidad de generalización.
        *   `random_state=0`: Asegura la reproducibilidad de los resultados.

    *   **Red Neuronal Profunda (MLP - construida con Keras)**:
        *   **Arquitectura**:
            *   Capa de entrada: Define la forma de los datos de entrada (igual al número de características del dataset: 256 para Original, 20 para PCA/UMAP).
            *   `Dense(128, activation='relu')`: Primera capa oculta con 128 neuronas y función de activación ReLU (Rectified Linear Unit), que introduce no linealidad.
            *   `Dropout(0.3)`: Técnica de regularización que desactiva aleatoriamente el 30% de las neuronas durante el entrenamiento para reducir el sobreajuste.
            *   `Dense(64, activation='relu')`: Segunda capa oculta con 64 neuronas y activación ReLU. Permite al modelo aprender representaciones más complejas.
            *   `Dense(10, activation='softmax')`: Capa de salida con 10 neuronas (una por cada dígito/clase) y función de activación Softmax, que produce una distribución de probabilidad sobre las 10 clases.
        *   **Configuración de Entrenamiento**:
            *   `optimizer=Adam(learning_rate=0.001)`: Algoritmo de optimización adaptativo que ajusta la tasa de aprendizaje durante el entrenamiento. Adam es una opción popular y eficiente. Se usa una tasa de aprendizaje inicial de 0.001.
            *   `loss='categorical_crossentropy'`: Función de pérdida adecuada para problemas de clasificación multiclase con etiquetas codificadas como one-hot (usando `to_categorical`).
            *   `metrics=['accuracy']`: Métrica para evaluar el rendimiento durante el entrenamiento y la evaluación.
            *   `epochs=20`: Número de pasadas completas sobre el conjunto de entrenamiento. 20 épocas es un valor relativamente bajo, elegido para un entrenamiento más rápido en este ejemplo, pero podría aumentarse si se observara subajuste.
            *   `batch_size=64`: Número de muestras por actualización de gradiente. Un tamaño de lote de 64 es común y equilibra la estabilidad del gradiente con la velocidad de entrenamiento.
            *   `validation_split=0.1`: Una fracción (10%) del conjunto de entrenamiento se reserva para validar el modelo durante el entrenamiento y monitorear el sobreajuste.
    """)

In [ ]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!mv cloudflared-linux-amd64 /usr/local/bin/cloudflared

#Ejecutar Streamlit
!streamlit run usps_dashboard.py &>/content/logs.txt & #Cambiar TAMExam1.py por el nombre de tu archivo principal

#Exponer el puerto 8501 con Cloudflare Tunnel
!cloudflared tunnel --url http://localhost:8501 > /content/cloudflared.log 2>&1 &

#Leer la URL pública generada por Cloudflare
import time
time.sleep(5)  # Esperar que se genere la URL

import re
found_context = False  # Indicador para saber si estamos en la sección correcta

with open('/content/cloudflared.log') as f:
    for line in f:
        #Detecta el inicio del contexto que nos interesa
        if "Your quick Tunnel has been created" in line:
            found_context = True

        #Busca una URL si ya se encontró el contexto relevante
        if found_context:
            match = re.search(r'https?://\S+', line)
            if match:
                url = match.group(0)  #Extrae la URL encontrada
                print(f'Tu aplicación está disponible en: {url}')
                break  #Termina el bucle después de encontrar la URL

In [ ]:
import os

res = input("Digite (1) para finalizar la ejecución del Dashboard: ")

if res.upper() == "1":
    # Find the process ID (PID) of the streamlit process
    # This assumes only one streamlit process is running
    try:
        # Use pgrep to find the PID of the streamlit process
        pid = os.popen("pgrep -f streamlit").read().strip()
        if pid:
            os.system(f"kill {pid}")  # Terminate the Streamlit process
            print(f"El proceso de Streamlit (PID: {pid}) ha sido finalizado.")
        else:
            print("No se encontró ningún proceso de Streamlit ejecutándose.")
    except Exception as e:
        print(f"Error al intentar finalizar el proceso de Streamlit: {e}")

    # Also try to stop the cloudflared tunnel process
    try:
        # Use pgrep to find the PID of the cloudflared process
        pid_cf = os.popen("pgrep -f cloudflared").read().strip()
        if pid_cf:
             # Use kill -9 for forceful termination
            os.system(f"kill -9 {pid_cf}")  # Terminate the cloudflared process
            print(f"El proceso de Cloudflared (PID: {pid_cf}) ha sido finalizado.")
        else:
            print("No se encontró ningún proceso de Cloudflared ejecutándose.")
    except Exception as e:
         print(f"Error al intentar finalizar el proceso de Cloudflared: {e}")

In [ ]:
!kill -9 <PID>